Implement DEAP (Distributed Evolutionary Algorithms) using Python. 


In [1]:
import random
from deap import base, creator, tools, algorithms

# --- Problem Setup ---
IND_SIZE = 10  # Size of individual (number of genes)
POP_SIZE = 50  # Population size
GENS = 20      # Number of generations

# --- Define Fitness & Individual ---
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximize
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# --- Gene Initialization ---
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# --- Evaluation Function ---
def eval_sum(individual):
    return (sum(individual),)  # Tuple needed for DEAP

toolbox.register("evaluate", eval_sum)
toolbox.register("mate", tools.cxTwoPoint)  # Crossover
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)  # Bit flip mutation
toolbox.register("select", tools.selTournament, tournsize=3)  # Selection

# --- Run Evolution ---
def main():
    pop = toolbox.population(n=POP_SIZE)

    # Statistics to track progress
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum(f[0] for f in x) / len(x))
    stats.register("max", lambda x: max(f[0] for f in x))

    # Run Genetic Algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=GENS,
                                   stats=stats, verbose=True)

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is:", best_ind)
    print("Fitness:", best_ind.fitness.values[0])

if __name__ == "__main__":
    main()


gen	nevals	avg 	max
0  	50    	4.82	7  
1  	29    	5.72	8  
2  	31    	6.5 	9  
3  	22    	7.28	10 
4  	30    	8.24	10 
5  	31    	8.9 	10 
6  	35    	9.26	10 
7  	34    	9.48	10 
8  	36    	9.78	10 
9  	27    	9.84	10 
10 	34    	9.96	10 
11 	29    	9.86	10 
12 	27    	9.82	10 
13 	28    	9.92	10 
14 	37    	9.78	10 
15 	23    	9.84	10 
16 	28    	9.82	10 
17 	34    	9.76	10 
18 	29    	9.7 	10 
19 	21    	9.72	10 
20 	24    	9.86	10 
Best individual is: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Fitness: 10.0


In [3]:
import random
from deap import creator, base, tools, algorithms

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalOneMax(individual):
    return sum(individual),

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=300)

NGEN=40
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10)
print(len(top10[0]))

100


In [4]:
import random
from deap import base, creator, tools

# Step 1: Create the problem environment
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximizing fitness
creator.create("Individual", list, fitness=creator.FitnessMax)

# Step 2: Define the fitness function
def evaluate(individual):
    # Count the number of 1's in the individual (this is the "fitness")
    return sum(individual),  # Return as a tuple (required by DEAP)

# Step 3: Set up the genetic algorithm components
def main():
    # Create an initial population of 50 individuals (each with 50 genes)
    population = [creator.Individual([random.randint(0, 1) for _ in range(10)]) for _ in range(50)]

    # Set up the toolbox with the genetic algorithm operations
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)  # Randomly generate 0 or 1
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_bool)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxTwoPoint)  # Two-point crossover (mixing two individuals)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Mutation: flip a bit with 5% chance
    toolbox.register("select", tools.selTournament, tournsize=3)  # Tournament selection
    toolbox.register("evaluate", evaluate)  # The function that evaluates fitness

    # Step 4: Run the algorithm for 10 generations
    for generation in range(20):  # Reduced to 10 generations
        print(f"Generation {generation}")

        # Step 5: Evaluate fitness of each individual
        for ind in population:
            if not ind.fitness.valid:
                ind.fitness.values = toolbox.evaluate(ind)

        # Step 6: Select the next generation (parents)
        offspring = toolbox.select(population, len(population))  # Select based on fitness
        offspring = list(map(toolbox.clone, offspring))  # Clone selected individuals to create offspring

        # Step 7: Apply crossover (mixing two individuals)
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.7:  # 70% chance to mate
                toolbox.mate(child1, child2)  # Perform crossover
                del child1.fitness.values  # Recalculate fitness after crossover
                del child2.fitness.values

        # Step 8: Apply mutation (randomly flip a bit)
        for mutant in offspring:
            if random.random() < 0.2:  # 20% chance to mutate
                toolbox.mutate(mutant)  # Perform mutation
                del mutant.fitness.values  # Recalculate fitness after mutation

        # Step 9: Recalculate fitness of the mutated offspring
        for ind in offspring:
            if not ind.fitness.valid:
                ind.fitness.values = toolbox.evaluate(ind)

        # Step 10: Replace the old population with the new offspring
        population[:] = offspring

        # Step 11: Print the best solution (highest number of 1's)
        best_individual = tools.selBest(population, 1)[0]
        print(f"Best individual: {best_individual}, Fitness: {best_individual.fitness.values[0]}")

if __name__ == "__main__":
    main()


Generation 0
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0], Fitness: 9.0
Generation 1
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 0, 1], Fitness: 9.0
Generation 2
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 3
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 4
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 5
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 6
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 7
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 8
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 9
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 10
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 11
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 12
Best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 10.0
Generation 

C:\Users\DELL\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\DELL\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
